In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=5,
    leverages={"APTUSDT-PERP.BINANCE": 5}
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="INFO"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, PositionClosed, OrderAccepted, OrderFilled
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig, frozen=True):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 10
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)

    

    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry(tick)
        self.check_for_exit()


    def check_for_entry(self, tick: TradeTick):
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )
        
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=quantity,
            )
            self.position_open_price = tick.price
            self.submit_order(order)
            

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)

        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    
    def on_position_opened(self, event: PositionOpened):
        self.position = self.cache.position(event.position_id)

        if self.position.side == PositionSide.LONG:
            order_side = OrderSide.BUY

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 - 0.1)
            )
            
        else:
            order_side = OrderSide.SELL

            limit_price = self.instrument.make_price(
                self.position_open_price * (1 + 0.1)
            )

        
        quantity = self.instrument.make_qty(
            self.config.trade_size
        )

        self.__limit_order = self.order_factory.limit(
            self.config.instrument_id,
            order_side,
            quantity,
            limit_price,
            reduce_only=False,
            post_only=False,
            time_in_force=TimeInForce.GTC,
            expire_time=None,
            quote_quantity=False,
            emulation_trigger=TriggerType.NO_TRIGGER,
            trigger_instrument_id=None,
            exec_algorithm_id=None,
            exec_algorithm_params=None,
            display_qty=None
        )


    
    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-18T15:49:14.228547832Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-18T15:49:14.228579748Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-18T15:49:14.228581508Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-18T15:49:14.228582364Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-18T15:49:14.228583276Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-18T15:49:14.228584268Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-18T15:49:14.228584917Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-18T15:49:14.228585805Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-18T15:49:14.228587282Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-04-18T15:49

In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

c_spawn_id=None, tags=None)
2024-12-10T00:04:47.157000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(SELL 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000447-001-000-827, venue_order_id=None, position_id=None, tags=None), position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000)
2024-12-10T00:04:47.157000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000447-001-000-827, account_id=BINANCE-001, ts_event=1733789087157000000)
2024-12-10T00:04:47.157000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:04:47.157000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=72.38965867 USDT, locked=0.59811270 USDT, free=71.79154597 USDT)], margins=[MarginBalance(initial=0.00000000 USDT

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,04bde7ac-62f3-47a3-889c-1dc857571337,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00
O-20241210-000008-001-000-3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-002,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,10.0,...,None,None,None,None,None,None,None,34e7a402-702f-4ae9-892b-384346afe4b8,2024-12-10 00:00:08.205000+00:00,2024-12-10 00:00:08.205000+00:00
O-20241210-000008-001-000-5,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-003,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-006,MARKET,BUY,10.0,...,None,None,None,None,None,None,None,955e9d73-9018-4740-a599-005be73e2131,2024-12-10 00:00:08.218000+00:00,2024-12-10 00:00:08.218000+00:00
O-20241210-000009-001-000-7,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-004,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-008,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,a9456758-0c0f-4313-81d7-ab08341bb34f,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00
O-20241210-000009-001-000-8,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-005,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-010,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,7d39861b-bc2a-444e-8f9d-9e72b12ccc02,2024-12-10 00:00:09.609000+00:00,2024-12-10 00:00:09.609000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241210-012537-001-000-3196,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-1977,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-3235,MARKET,BUY,10.0,...,None,None,None,None,None,None,None,02648374-4eb3-46a5-8c0b-2858f1ac5936,2024-12-10 01:25:37.481000+00:00,2024-12-10 01:25:37.481000+00:00
O-20241210-012538-001-000-3197,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-1978,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-3237,MARKET,BUY,10.0,...,None,None,None,None,None,None,None,353125c5-2b2d-4471-924a-d67458e2d4bb,2024-12-10 01:25:38.552000+00:00,2024-12-10 01:25:38.552000+00:00
O-20241210-012541-001-000-3199,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-1979,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-3239,MARKET,SELL,10.0,...,None,None,None,None,None,None,None,937da89c-5363-4ed7-84a7-8b0567437cb6,2024-12-10 01:25:41.238000+00:00,2024-12-10 01:25:41.238000+00:00


0-001-000-857, account_id=BINANCE-001, reason='REDUCE_ONLY MARKET BUY order would have increased position', ts_event=1733789100823000000)
2024-12-10T00:05:00.825000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000500-001-000-858, side=BUY, type=MARKET, quantity=10.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T00:05:00.825000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(BUY 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000500-001-000-858, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:05:00.825000000Z [INFO

In [10]:
engine.trader.generate_positions_report()

=1733789102198000000)
2024-12-10T00:05:02.198000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE net_position=0
2024-12-10T00:05:02.198000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_maint=0.00000000 USDT
2024-12-10T00:05:02.198000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] PositionClosed(instrument_id=APTUSDT-PERP.BINANCE, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, account_id=BINANCE-001, opening_order_id=O-20241210-000501-001-000-865, closing_order_id=O-20241210-000502-001-000-867, entry=BUY, side=FLAT, signed_qty=0.0, quantity=0.0, peak_qty=10.0, currency=USDT, avg_px_open=11.748676999999999, avg_px_close=11.7461, realized_return=-0.00022, realized_pnl=-0.14324389 USDT, unrealized_pnl=0.00000000 USDT, ts_opened=1733789101760000000, ts_last=1733789102198000000, ts_closed=1733789102198000000, duration_ns=438000000)
2024-12-10T00:05:02.201000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000-0ec3739e-3f40-4f27-8b2e-8338bd04577a,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,O-20241210-000008-001-000-3,SELL,FLAT,0.0,10.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788808205000000,2024-12-10 00:00:08.204999936+00:00,3.157000e+09,11.935610,11.928018,[0.11931814 USDT],0.00064,-0.04339814 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-57faf6fc-24d7-4f3f-862d-3fa5e179ace3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000008-001-000-5,O-20241210-000009-001-000-7,BUY,FLAT,0.0,10.0,1733788808218000000,2024-12-10 00:00:08.218000+00:00,1733788809609000000,2024-12-10 00:00:09.608999936+00:00,1.391000e+09,11.928191,11.929914,[0.11929053 USDT],0.00014,-0.10206053 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-ff74aae4-2158-4d4f-baaa-89144a29e867,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000009-001-000-8,O-20241210-000010-001-000-10,SELL,FLAT,0.0,10.0,1733788809609000000,2024-12-10 00:00:09.609000+00:00,1733788810101000000,2024-12-10 00:00:10.100999936+00:00,4.920000e+08,11.929809,11.930600,[0.11930205 USDT],-0.00007,-0.12721205 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-151e7268-b998-4730-a3cb-971cef989251,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-12,O-20241210-000010-001-000-14,BUY,FLAT,0.0,10.0,1733788810101000000,2024-12-10 00:00:10.101000+00:00,1733788810163000000,2024-12-10 00:00:10.163000064+00:00,6.200000e+07,11.928200,11.920700,[0.11924450 USDT],-0.00063,-0.19424450 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-9492b676-15ac-4300-ab64-cb0446302e6f,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000010-001-000-15,O-20241210-000010-001-000-17,SELL,FLAT,0.0,10.0,1733788810181000000,2024-12-10 00:00:10.181000+00:00,1733788810201000000,2024-12-10 00:00:10.200999936+00:00,2.000000e+07,11.919300,11.921500,[0.11920400 USDT],-0.00018,-0.14120400 USDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APTUSDT-PERP.BINANCE-MACDStrategy-000-9a6f5a84-76e7-4d40-919b-f7f56cbe0241,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-012526-001-000-3188,O-20241210-012527-001-000-3190,BUY,FLAT,0.0,10.0,1733793926981000000,2024-12-10 01:25:26.981000+00:00,1733793927432000000,2024-12-10 01:25:27.432000+00:00,4.510000e+08,11.877900,11.876500,[0.11877200 USDT],-0.00012,-0.13277200 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-bb55334a-d9d0-4355-8b0e-3fcc9c5960e4,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-012527-001-000-3191,O-20241210-012532-001-000-3193,BUY,FLAT,0.0,10.0,1733793927452000000,2024-12-10 01:25:27.452000+00:00,1733793932742000000,2024-12-10 01:25:32.742000128+00:00,5.290000e+09,11.878491,11.883529,[0.11881011 USDT],0.00042,-0.06843011 USDT
APTUSDT-PERP.BINANCE-MACDStrategy-000-3acc81b1-e2ba-472a-887e-f2c26aceec18,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-012532-001-000-3194,O-20241210-012537-001-000-3196,SELL,FLAT,0.0,10.0,1733793932854000000,2024-12-10 01:25:32.854000+00:00,1733793937481000000,2024-12-10 01:25:37.480999936+00:00,4.627000e+09,11.883209,11.882891,[0.11883051 USDT],0.00003,-0.11565051 USDT


 BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(BUY 10.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-000512-001-000-906, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:05:12.520000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000512-001-000-906, account_id=BINANCE-001, ts_event=1733789112520000000)
2024-12-10T00:05:12.520000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:05:12.520000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=69.89434394 USDT, locked=0.00000000 USDT, free=69.89434394 USDT)], margins=[], event_id=4b59b92c-fccf-489d-84b6-399ef1715353)
2024-12-10T00:05:12.520000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFille

In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

realized_pnl=0.10200000 USDT, ts_opened=1733789117580000000, ts_last=1733789120947000000, ts_closed=0, duration_ns=0)
2024-12-10T00:05:20.947000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:05:20.947000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=69.82006785 USDT, locked=0.50921511 USDT, free=69.31085274 USDT)], margins=[MarginBalance(initial=0.00000000 USDT, maintenance=0.50921511 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=89c5f1da-36de-42ca-9d2e-4fd5401ce539)
2024-12-10T00:05:20.947000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000520-001-000-915, venue_order_id=BINANCE-1-566, account_id=BINANCE-001, trade_id=BINANCE-1-869, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=SELL, order_type=MARKET, last

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,99.94032195,0.06087207,99.87944988,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:08.205000+00:00,99.95381915,0.60871560,99.34510355,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 00:00:08.205000+00:00,99.95660186,0.10956881,99.84703305,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 01:25:38.552000+00:00,0.72258465,0.09090572,0.63167893,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 01:25:41.238000+00:00,0.71926857,0.60604320,0.11322537,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 01:25:41.238000+00:00,0.66637712,0.56968061,0.09669651,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}
2024-12-10 01:25:41.238000+00:00,0.66102954,0E-8,0.66102954,USDT,BINANCE-001,MARGIN,USDT,[],False,{}


1733789133837000000, ts_closed=0, duration_ns=0)
2024-12-10T00:05:33.837000000Z [INFO] BACKTESTER-001.Portfolio: APTUSDT-PERP.BINANCE margin_init=0.00000000 USDT
2024-12-10T00:05:33.837000000Z [INFO] BACKTESTER-001.Portfolio: Updated AccountState(account_id=BINANCE-001, account_type=MARGIN, base_currency=USDT, is_reported=False, balances=[AccountBalance(total=69.34589322 USDT, locked=0.50949689 USDT, free=68.83639633 USDT)], margins=[MarginBalance(initial=0.00000000 USDT, maintenance=0.50949689 USDT, instrument_id=APTUSDT-PERP.BINANCE)], event_id=68192c54-1512-486d-b5c2-372ca548ec22)
2024-12-10T00:05:33.837000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderFilled(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-000533-001-000-933, venue_order_id=BINANCE-1-578, account_id=BINANCE-001, trade_id=BINANCE-1-891, position_id=APTUSDT-PERP.BINANCE-MACDStrategy-000, order_side=SELL, order_type=MARKET, last_qty=8.5, last_px=11.76330 USDT, commission=0.04999403 USDT, liquidit